# Subjectivity classification with CNNs

In this notebook we implement the approched described in this [paper](https://arxiv.org/pdf/1408.5882.pdf) for classifiying sentences using Convolutional Neural Networks. In particular, we will classify sentences into "subjective" or "objective". 

## Subjectivity Dataset

The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [1]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [2]:
 unpack_dataset()

--2020-03-02 10:37:08--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz.1’

rotten_imdb.tar.gz. 100%[===================>] 507.42K   493KB/s    in 1.0s    

2020-03-02 10:37:09 (493 KB/s) - ‘rotten_imdb.tar.gz.1’ saved [519599/519599]

mkdir: data: File exists
x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [3]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/glove.6B.300d.txt'),
 PosixPath('data/glove.6B.100d.txt'),
 PosixPath('data/glove.6B.50d.txt'),
 PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000'),
 PosixPath('data/glove.6B.200d.txt'),
 PosixPath('data/glove.6B.zip')]

From the readme file:
- quote.tok.gt9.5000 contains 5000 subjective sentences (or snippets)
- plot.tok.gt9.5000 contains 5000 objective sentences

In [4]:
! head data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 
spurning her mother's insistence that she get on with her life , mary is thrown out of the house , rejected by joe , and expelled from school as she grows larger with child . 
amitabh can't believe the board of directors and his mind is filled with revenge and what better revenge than robbing the bank himself , ironic as it may sound . 
she , among others excentricities , talks to a small rock , gertrude , like if she was alive . 
this gives the girls a fair chance of pulling the wool over their eyes using their sexiness to poach any last vestige of common sense the dons might have had . 
styled after vh1's " behin

## String cleaning functions

In [5]:
import numpy as np
from collections import defaultdict
import re

In [6]:
def read_file(path):
    """ Read file returns a shuttled list.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = np.array(f.readlines())
    return content

In [7]:
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = line.strip()
            words = set(line.split())
            for word in words:
                vocab[word] += 1
    return vocab       

## Split train and test

In [8]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip() for line in sub_content])
obj_content = np.array([line.strip() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

In [12]:
# getting vocab from training sets
data_vocab = get_vocab([X_train])

In [13]:
#data_vocab

## Embedding Layer

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# an Embedding module containing 10 (words) tensors of size 3
embed = nn.Embedding(10, 3)
a = torch.LongTensor([[1,2,4,5,1]])
embed(a)

tensor([[[-0.8965, -1.7041, -1.2807],
         [ 0.4190, -0.9627, -1.1611],
         [ 0.2568, -0.2770, -0.4655],
         [ 0.2099,  1.0432, -2.6004],
         [-0.8965, -1.7041, -1.2807]]], grad_fn=<EmbeddingBackward>)

In [16]:
## here is the randomly initialized embeddings
embed.weight.data

tensor([[ 0.0553,  0.7693, -1.3115],
        [-0.8965, -1.7041, -1.2807],
        [ 0.4190, -0.9627, -1.1611],
        [-0.1029,  1.2430, -1.3690],
        [ 0.2568, -0.2770, -0.4655],
        [ 0.2099,  1.0432, -2.6004],
        [ 1.5343,  1.5588, -1.4683],
        [-0.9402, -1.9589, -1.0073],
        [-0.3618,  0.6040,  0.6999],
        [-0.1928,  0.2359, -0.0965]])

### Initializing embedding layer with Glove embeddings

To get glove pre-trained embeddings:
    `wget http://nlp.stanford.edu/data/glove.6B.zip`

In [17]:
def unpack_glove():
    ! wget http://nlp.stanford.edu/data/glove.6B.zip
    ! mkdir data
    ! unzip glove.6B.zip -C data
    
# unpack_glove()

In this section I am keeping the whole Glove embeddings. You can decide to keep just the words on your training set.

In [18]:
! head -2 data/glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392


We would like to initialize the embeddings from our model with the pre-trained Glove embeddings. After initializing we should "freeze" the embeddings at least initially. The rationale is that we first want the network to learn weights for the other parameters that were randomly initialize. After that phase we could finetune the embeddings to our task. 

`embed.weight.requires_grad = False` freezes the embedding parameters.

The following code initializes the embedding. Here `V` is the vocabulary size and `D` is the embedding size. `pretrained_weight` is a numpy matrix of shape `(V, D)`.

In [19]:
def loadGloveModel(gloveFile=PATH/"glove.6B.300d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [20]:
word_vecs = loadGloveModel()

In [21]:
print(len(word_vecs.keys()), len(data_vocab.keys()))

400000 21416


In [22]:
def delete_rare_words(word_vecs, data_vocab, min_df=2):
    """ Deletes rare words from data_vocab
    
    Deletes words from data_vocab if they are not in word_vecs
    and don't have at least min_df occurrencies in data_vocab.
    """
    words_delete = []
    for word in data_vocab:
        if data_vocab[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: data_vocab.pop(word)
    return data_vocab

In [23]:
len(data_vocab.keys())

21416

In [24]:
# clean up issues here
data_vocab = delete_rare_words(word_vecs, data_vocab, min_df=2) 

In [25]:
len(data_vocab.keys())

18756

In [26]:
def create_embedding_matrix(word_vecs, data_vocab, min_df=2, D=300):
    """Creates embedding matrix from word vectors. """
    data_vocab = delete_rare_words(word_vecs, data_vocab, min_df)
    V = len(data_vocab.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, D), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(D, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, D)
    vocab2index["UNK"] = 1
    i = 2
    for word in data_vocab:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25,D)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [27]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, data_vocab)

In [28]:
len(pretrained_weight) # note that index 0 is for padding

18758

In [29]:
D = 300
V = len(pretrained_weight)
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0900,  0.0625,  0.1929,  ..., -0.1001,  0.2241, -0.0245],
        [-0.0977, -0.5308, -0.2896,  ...,  0.4010, -0.1188,  0.1739],
        ...,
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

Question: How many parameters do we have in this embedding matrix?

## Encoding training and validation sets

We will be using 1D Convolutional neural networks as our model. CNNs assume a fixed input size so we need to assume a fixed size and truncate or pad the sentences as needed. Let's find a good value to set our sequence length to.

In [30]:
x_len = np.array([len(x.split()) for x in X_train])

In [31]:
np.percentile(x_len, 95) # let set the max sequence len to N=40

43.0

In [32]:
X_train[0]

'will god let her fall or give her a new path ?'

In [33]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("will", vocab2index["UNK"])

10

In [34]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([10,  2,  6,  3,  5, 11,  8,  3,  9, 12,  7,  4])

In [35]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [36]:
encode_sentence(X_train[0])

array([10,  2,  6,  3,  5, 11,  8,  3,  9, 12,  7,  4,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [37]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [38]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Playing and debugging CNN layers

In [48]:
V = len(pretrained_weight)
emb_size = 300
N = 40

In [49]:
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0900,  0.0625,  0.1929,  ..., -0.1001,  0.2241, -0.0245],
        [-0.0977, -0.5308, -0.2896,  ...,  0.4010, -0.1188,  0.1739],
        ...,
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

In [85]:
x = x_train[:2]
x.shape

(2, 40)

In [86]:
x = torch.LongTensor(x)
x

tensor([[10,  2,  6,  3,  5, 11,  8,  3,  9, 12,  7,  4,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [23, 20, 19, 15, 24, 16, 29, 25, 17, 14, 33, 30, 26, 34, 16, 13, 27, 31,
         35, 22, 28, 36, 21, 18, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]])

In [87]:
x = emb(x)
x.shape

torch.Size([2, 40, 300])

In [88]:
x = x.transpose(1,2)  # needs to convert x to (batch, embedding_dim, sentence_len)
x.size()

torch.Size([2, 300, 40])

In [89]:
conv1 = nn.Conv1d(in_channels=emb_size, out_channels=100, kernel_size=3)
conv2 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3)
conv3 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3)
batch1 = nn.BatchNorm1d(100)
batch2 = nn.BatchNorm1d(100)
batch3 = nn.BatchNorm1d(100)

In [90]:
x = F.elu(conv1(x))
x = batch1(x)
x.size()

torch.Size([2, 100, 38])

In [91]:
x = F.elu(conv2(x))
x = batch2(x)
x.size()

torch.Size([2, 100, 36])

In [92]:
x = F.elu(conv3(x))
x = batch3(x)
x.size()

torch.Size([2, 100, 34])

In [80]:
x = F.avg_pool1d(x, 34)
x.size()

torch.Size([2, 100, 1])

In [81]:
x = x.squeeze(dim=2)
x.size()

torch.Size([2, 100])

After this we have a fully connected network. Let's write a network that implements this.

## 1D CNN model for sentence classification

Notation:
* V -- vocabulary size
* D -- embedding size
* N -- MAX Sentence length

In [122]:
class SentenceCNN(nn.Module):
    
    def __init__(self, V, emb_size, glove_weights=None):
        super(SentenceCNN, self).__init__()
        self.embedding = nn.Embedding(V, emb_size, padding_idx=0)
        if glove_weights is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embedding.weight.requires_grad = False ## freeze embeddings

        self.conv1 = nn.Conv1d(in_channels=emb_size, out_channels=100, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3)
        
        self.bn1 = nn.BatchNorm1d(100)
        self.bn2 = nn.BatchNorm1d(100)
        self.bn3 = nn.BatchNorm1d(100)
        
        self.fc = nn.Linear(100, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = F.dropout(x, 0.1)
        x = x.transpose(1,2)
        
        # convolutional network
        x = self.bn1(F.elu(self.conv1(x)))
        x = self.bn2(F.elu(self.conv2(x)))
        x = self.bn3(F.elu(self.conv3(x)))
        
        x = F.avg_pool1d(x, 34).squeeze(dim=2)
        return self.fc(x)   

In [123]:
V = len(pretrained_weight)
emb_size = 300
N = 40
model = SentenceCNN(V, D, glove_weights=pretrained_weight)

In [98]:
# testing the model
x = x_train[:10]
print(x.shape)
x = torch.LongTensor(x)

(10, 40)


In [99]:
y_hat = model(x)
y_hat.size()

torch.Size([10, 1])

## Training

Note that I am not bodering with mini-batches since our dataset is small.

In [100]:
model = SentenceCNN(V, emb_size, glove_weights=pretrained_weight) #.cuda()

In [101]:
def val_metrics(m):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    y_hat = m(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [102]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.69488126039505, 0.49399998784065247)

In [103]:
# this filters parameters with p.requires_grad=True
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [130]:
def set_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [132]:
def train_epocs(model, optimizer, epochs=10):
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, accuracy = val_metrics(model)
        print("train loss %.3f test loss %.3f and accuracy %.3f" % 
              (loss.item(), val_loss, accuracy))

In [133]:
model = SentenceCNN(V, D, glove_weights=pretrained_weight)
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [134]:
train_epocs(model, optimizer, epochs=10)

train loss 0.702 test loss 0.671 and accuracy 0.719
train loss 0.555 test loss 0.354 and accuracy 0.851
train loss 0.341 test loss 0.698 and accuracy 0.779
train loss 0.327 test loss 0.794 and accuracy 0.782
train loss 0.325 test loss 0.525 and accuracy 0.845
train loss 0.305 test loss 0.395 and accuracy 0.877
train loss 0.293 test loss 0.441 and accuracy 0.865
train loss 0.289 test loss 0.486 and accuracy 0.850
train loss 0.287 test loss 0.463 and accuracy 0.859
train loss 0.281 test loss 0.407 and accuracy 0.871


In [135]:
set_learning_rate(optimizer, 0.001)

In [136]:
train_epocs(model, optimizer, epochs=5)

train loss 0.276 test loss 0.378 and accuracy 0.868
train loss 0.275 test loss 0.357 and accuracy 0.874
train loss 0.274 test loss 0.336 and accuracy 0.875
train loss 0.273 test loss 0.325 and accuracy 0.878
train loss 0.273 test loss 0.313 and accuracy 0.879


In [137]:
# how to figure out the parameters
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 100, 3]), torch.Size([100]), torch.Size([100, 100, 3]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([1, 100]), torch.Size([1])]


### Unfreezing the embeddings

In [138]:
# unfreezing the embeddings
model.embedding.weight.requires_grad = True

In [139]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([18758, 300]), torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 100, 3]), torch.Size([100]), torch.Size([100, 100, 3]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([100]), torch.Size([1, 100]), torch.Size([1])]


In [141]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [143]:
train_epocs(model, optimizer, epochs=10)

train loss 0.272 test loss 0.292 and accuracy 0.889
train loss 0.259 test loss 0.284 and accuracy 0.888
train loss 0.248 test loss 0.287 and accuracy 0.887
train loss 0.238 test loss 0.285 and accuracy 0.883
train loss 0.226 test loss 0.267 and accuracy 0.890
train loss 0.216 test loss 0.259 and accuracy 0.900
train loss 0.205 test loss 0.259 and accuracy 0.900
train loss 0.196 test loss 0.255 and accuracy 0.902
train loss 0.185 test loss 0.256 and accuracy 0.898
train loss 0.178 test loss 0.252 and accuracy 0.905


## Whithout pretrain emmbeddings

In [144]:
V = len(pretrained_weight)
model = SentenceCNN(V, emb_size=300)

In [146]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [147]:
train_epocs(model, optimizer, epochs=10)

train loss 0.692 test loss 0.665 and accuracy 0.585
train loss 0.717 test loss 0.745 and accuracy 0.563
train loss 0.568 test loss 0.489 and accuracy 0.769
train loss 0.465 test loss 0.563 and accuracy 0.756
train loss 0.430 test loss 0.527 and accuracy 0.786
train loss 0.396 test loss 0.470 and accuracy 0.816
train loss 0.351 test loss 0.485 and accuracy 0.836
train loss 0.316 test loss 0.485 and accuracy 0.836
train loss 0.279 test loss 0.481 and accuracy 0.846
train loss 0.238 test loss 0.515 and accuracy 0.843


In [148]:
set_learning_rate(optimizer, lr=0.001)

In [149]:
train_epocs(model, optimizer)

train loss 0.205 test loss 0.452 and accuracy 0.848
train loss 0.204 test loss 0.409 and accuracy 0.848
train loss 0.199 test loss 0.374 and accuracy 0.858
train loss 0.195 test loss 0.355 and accuracy 0.858
train loss 0.193 test loss 0.350 and accuracy 0.863
train loss 0.189 test loss 0.336 and accuracy 0.860
train loss 0.185 test loss 0.318 and accuracy 0.868
train loss 0.180 test loss 0.308 and accuracy 0.868
train loss 0.177 test loss 0.314 and accuracy 0.864
train loss 0.175 test loss 0.313 and accuracy 0.867


In [150]:
train_epocs(model, optimizer)

train loss 0.169 test loss 0.304 and accuracy 0.867
train loss 0.166 test loss 0.299 and accuracy 0.867
train loss 0.162 test loss 0.301 and accuracy 0.865
train loss 0.159 test loss 0.304 and accuracy 0.872
train loss 0.157 test loss 0.292 and accuracy 0.872
train loss 0.155 test loss 0.294 and accuracy 0.877
train loss 0.151 test loss 0.295 and accuracy 0.874
train loss 0.147 test loss 0.296 and accuracy 0.875
train loss 0.143 test loss 0.299 and accuracy 0.869
train loss 0.141 test loss 0.294 and accuracy 0.880


In [151]:
train_epocs(model, optimizer)

train loss 0.138 test loss 0.295 and accuracy 0.880
train loss 0.133 test loss 0.298 and accuracy 0.874
train loss 0.132 test loss 0.294 and accuracy 0.876
train loss 0.128 test loss 0.294 and accuracy 0.874
train loss 0.126 test loss 0.291 and accuracy 0.882
train loss 0.123 test loss 0.294 and accuracy 0.874
train loss 0.120 test loss 0.293 and accuracy 0.874
train loss 0.119 test loss 0.285 and accuracy 0.880
train loss 0.114 test loss 0.288 and accuracy 0.877
train loss 0.112 test loss 0.293 and accuracy 0.874
